In [4]:
from azure.identity import AzureCliCredential
from azure.core.credentials import TokenCredential, AccessToken
from requests import get
import time
import logging
logger = logging.getLogger(__name__)

def calc_time(string):
    global start_time

    if string == "start":
        start_time = time.time()
    else:
        end_time = time.time()
        return  end_time - start_time


class CachedCredential(TokenCredential):
  def __init__(self, delegate: TokenCredential, logger) -> None:
    self.delegate = delegate
    self.logger = logger
    self._token : dict[str, AccessToken] = {}

  def get_token(self, scope: str, **kwargs) -> AccessToken:
    token = self._token.get(scope)
    if not token or token.expiry < time.time():
      calc_time("start")
      self._token[scope] = token = self.delegate.get_token(scope, **kwargs)
      elapsed_time = calc_time("end")
      self.logger.info(
            f"Time taken to generate token(CachedCredential) is {elapsed_time:.2f} seconds."
        )
    else:
        self.logger.info(
            f"Valid token exists"
        )
    return token


cachedCredential = CachedCredential(AzureCliCredential(), logger)

def request_fmc_token(organization_name, stage='prod'):
    """
    Get fmc token via AzureCliCredential. (Requires az login beforehand).

    organization_name e.g. nrcs-2-pf, ford-dat-3, uss-gen-6-pf
    """
    return cachedCredential.get_token(f'api://api-data-loop-platform-{organization_name}-{stage}/.default').token


def get_sequence(sequence_id, organization_name, fmc_token):
    """
    Does get sequence Rest call for sequence_id. Returns sequence.
    """
    fmc_headers = {
        'Cache-Control': 'no-cache',
        'Authorization': f'Bearer {fmc_token}',
        'Origin': 'https://developer.bosch-data-loop.com'
    }
    url = f'https://api.azr.bosch-data-loop.com/measurement-data-processing/v3/organizations/{organization_name}/sequence/{sequence_id}'
    response = get(url, headers=fmc_headers)
    if response.status_code == 200:
        sequence = response.json()
        return sequence
    else:
        logger.error(f'Get sequence call to FMC failed. status_code: {response.status_code}, reason: {response.reason}, url: {url}')
        return None


def get_sequences(fmc_query, organization_name, fmc_token):
    """
    Does get sequences Rest call for fmc query. Returns list of sequences.
    """
    fmc_headers = {
        'Cache-Control': 'no-cache',
        'Authorization': f'Bearer {fmc_token}',
        'Origin': 'https://developer.bosch-data-loop.com'
    }

    sequences = []

    items_per_page = 1000

    is_there_more_sequences = True
    page_index = 0
    while is_there_more_sequences:
        url = f'https://api.azr.bosch-data-loop.com/measurement-data-processing/v3/organizations/{organization_name}/sequence?itemsPerPage={items_per_page}&pageIndex={page_index}&filterQuery={fmc_query}'  # noqa: E501
        response = get(url, headers=fmc_headers)
        if response.status_code == 200:
            response_sequences = response.json()
            sequences.extend(response_sequences)
            if len(response_sequences) < items_per_page:
                is_there_more_sequences = False
        else:
            logger.error(f'Get sequences call to FMC failed. status_code: {response.status_code}, reason: {response.reason}, url: {url}')
            is_there_more_sequences = False
        page_index += 1
        print(f'FMC query at {page_index=}')

    return sequences

In [5]:
# first given a fmc query, find all checksums

organization_name = 'nrcs-2-pf'
fmc_token = request_fmc_token(organization_name)
fmc_query = 'Car.licensePlate = "LBXQ6155" and Sequence.recordingDate > "2025-02-09" and ReferenceFile.type = "PCAP" and ReferenceFile.type = "JSON_METADATA"'
# TODO limit 10
fmc_sequences = get_sequences(fmc_query, organization_name, fmc_token)[:10]


FMC query at page_index=1
FMC query at page_index=2
FMC query at page_index=3
FMC query at page_index=4


In [6]:
from glob import glob

dyperexprod_dir = '/home/jovyan/data/ReadOnly/dyperexprod/nrcs-2-pf'

checksum_to_fmc_id = {}
for sequence in fmc_sequences:
    checksum = None
    for meas_file in sequence['measurementFiles']:
        if 'bytesoup' in meas_file['path']:
            checksum = meas_file['checksum']

    if checksum is not None:
        checksum_to_fmc_id[checksum] = sequence['id']
    else:
        print(f'Failed to find checksum for id', sequence['id'])


In [ ]:
import os
from datetime import datetime
import json
import math

bot_username = 'autogen'
current_date_str = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

def get_latest_bolf(bolfs):
    latest_bolf = None
    latest_bolf_time = 0

    for bolf_path in bolfs:
        filename = os.path.basename(bolf_path)
        if bot_username in filename:
            # exclude auto generated files
            continue

        creation_date = os.path.basename(os.path.dirname(bolf_path))
        epoch = datetime.strptime(creation_date, "%Y_%m_%d_%H_%M_%S").timestamp()
        if epoch >= latest_bolf_time:
            epoch = latest_bolf_time
            latest_bolf = bolf_path
    return latest_bolf

expected_obstacle_type = 'curbstone_real_normal'
expected_obstacle_height = '0.16'
new_obstacle_height = '99'

for checksum in checksum_to_fmc_id:
    dev_bolfs = glob(f'/home/jovyan/data/ReadOnly/dypersiadev/nrcs-2-pf/{checksum}/processed_lidar/*/*.json')
    qua_bolfs = glob(f'/home/jovyan/data/ReadOnly/dypersiaqua/nrcs-2-pf/{checksum}/processed_lidar/*/*.json')
    bolfs = dev_bolfs + qua_bolfs

    latest_bolf = get_latest_bolf(bolfs)
    if not latest_bolf:
        print(f'Did not find latest bolf for {checksum=}')
        continue

    orig_bolf_filename = os.path.basename(latest_bolf)

    with open(latest_bolf) as f:
        bolf = json.loads(f.read())
    
    obstacle_type = list(bolf['openlabel']['objects'].values())[0]['type']

    if obstacle_type != expected_obstacle_type:
        continue

    for frame in bolf['openlabel']['frames'].values():
        for object in frame['objects'].values():
            height_attribute = object['object_data']['poly3d'][0]['attributes']['num'][0]
            if height_attribute['name'] != 'height':
                print('Warning: found non height attribute.')
                continue
            current_height_val = height_attribute['val']
            if type(current_height_val) == str:
                if current_height_val != expected_obstacle_height:
                    continue
                height_attribute['val'] = new_obstacle_height
            elif type(current_height_val) in [float, int]:
                if not math.isclose(expected_obstacle_height, float(new_obstacle_height), 0.001):
                    continue
                height_attribute['val'] = float(new_obstacle_height)

    new_bolf_name = f'{checksum}_{bot_username}_{bot_username}_{current_date_str}.json'

    with open(new_bolf_name, 'w') as f:
        f.write(json.dumps(bolf))


Did not find latest bolf for checksum='6fa97450f15cf9f7f22a0f22e3dcdf1d4e8b704ea93ed9565fae102332858920'
Did not find latest bolf for checksum='e4ac6cb3a57d7a92163d9e59165554d50febff4c3247c217b324bdf7517f7466'
end
Did not find latest bolf for checksum='c2120bab61ec90c17632a3438a80556260375af6226c17b2b94ebd04fd53eec0'
Did not find latest bolf for checksum='5c01f1c3ab6baa4d13e15c0f3ef8ec89cb6fc0f56ebf847882fd41b97f950aeb'
end
end
Did not find latest bolf for checksum='99594b6ac220f733c9a4aaed5546a41fd230071e67e4fb2c9d0b38ac2a2235e5'
Did not find latest bolf for checksum='9349afa1c79c4324abb064e749b6acca1ef84af069618e3bb4ca6dc33a0561e3'
Did not find latest bolf for checksum='8595142b604a0aca6028618042b1e261f1d7bf15cf27bf0d0c76c1d24db2b595'
